In [ ]:
from os.path import isdir, join
from os import listdir
import numpy as np
from warnings import warn
from random import shuffle

import menpo.io as mio
import dlib
from menpodetect import load_dlib_frontal_face_detector
from menpodetect import load_opencv_frontal_face_detector
from menpodetect.ffld2 import load_ffld2_frontal_face_detector
from menpofit.visualize import print_progress

try:
    %matplotlib inline
except NameError:
    pass

In [ ]:
def get_train_images(img_p, det_p):
    assert(isdir(img_p) and isdir(det_p))
    train_images = []
    ps = list(mio.image_paths(img_p))
    for imn in print_progress(ps):
        im = mio.import_image(imn)
        try:
            ln = mio.import_landmark_file(det_p + im.path.stem + '.pts')
        except ValueError:
            warn('The image {} is missing.'.format(im.path))
            continue
#             raise ValueError('The image {} is missing.'.format(im.path.stem))
        if not (ln.lms.n_points == 4):
            print(str(ln.path))
            continue
        im.landmarks['bb'] = ln
        train_images.append(im)
    return train_images

In [ ]:
detection_method = 'ffld2'
detection_glob = detection_method + '_*'

p0 = '/vol/atlas/homes/grigoris/misc/2016_ijcv/data/alignment_training/'
det_p = join('/vol/atlas/homes/grigoris/misc/2016_ijcv/data/alignment_training_bb/', detection_method, '')
assert(isdir(p0) and isdir(det_p))
path_pickles = '/vol/atlas/homes/grigoris/misc/2016_ijcv/data/pickles/'
assert(isdir(path_pickles))

In [ ]:
folds = ['ibug', 'afw', '300w', 'helen/trainset', 'helen/testset', 'lfpw/trainset', 'lfpw/testset']
images = []
for fold in folds:
    print(fold)
    im1 = get_train_images(p0 + fold + '/', det_p + fold + '/')
    assert(len(im1) > 50)
    images += im1

# Train Ensemble Regression Trees (One millisecond Face Alignment paper)

In [ ]:
from menpofit.dlib import DlibERT

model = DlibERT(images, group='PTS', bounding_box_group_glob='bb*',
                scales=(1.0,), n_perturbations=0, n_dlib_perturbations=2, 
                n_iterations=14, verbose=True)

In [ ]:
# from menpowidgets import visualize_images
# visualize_images(images)

In [ ]:
p_save_model = path_pickles + 'modelln_' + detection_method + '_dlibERT.model'
# mio.export_pickle(model, p_save_model)

m = model.algorithms[0].dlib_model
m.save(p_save_model)

# GN-DPM (patch aam) 

In [ ]:
# from menpo.feature import fast_dsift
# from menpofit.aam import PatchAAM, LucasKanadeAAMFitter

# features = fast_dsift
# patch_shape = (18, 18)
# crop = 0.2
# diagonal = 180
# # n_shape=[4, 15]
# # n_appearance=[60, 150]

# aam = PatchAAM(images, verbose=True, holistic_features=features, patch_shape=patch_shape,
#                diagonal=diagonal, scales=(1.), group='PTS')


In [ ]:
def return_detector(detection):
    if detection == 'dlib':
        return load_dlib_frontal_face_detector()
    elif detection == 'opencv':
        from functools import partial
        det = load_opencv_frontal_face_detector()
        return partial(det, min_neighbours=3)
    elif detection == 'ffld2':
        return load_ffld2_frontal_face_detector()
    else:
        raise RuntimeError('Not a valid choice of detection ({}).'.format(detection))
        
detector = return_detector(detection_method)

im = mio.import_builtin_asset.breakingbad_jpg()
del im.landmarks['PTS']
detector(im, group_prefix='bb')

# im.view_landmarks()
ll = model.fit_from_bb(im, im.landmarks['bb_0'].lms, max_iters=40)
im2 = ll.fitted_image
im2 = im2.crop_to_landmarks_proportion(0.3, group='final')
im2.view_landmarks(group='final')


In [ ]:
# im = mio.import_builtin_asset.breakingbad_jpg()
# im = im.crop_to_landmarks_proportion(0.3)
# random_rotations = np.random.randint(-20, 20, size=12)
# rotated_images = [im.rotate_ccw_about_centre(r) for r in random_rotations]
# for im_r in rotated_images:
#     detector(im_r)
#     try:
#         ll = model.fit_from_bb(im, im.landmarks['dlib_0'].lms, max_iters=40)
#     except KeyError:
#         im_r.view_landmarks(group='PTS', new_figure=True)
#         continue
#     im2 = ll.fitted_image
#     im2 = im2.crop_to_landmarks_proportion(0.3, group='final')
#     im2.view_landmarks(group='final', new_figure=True)
# #     im_r.view_landmarks(group='PTS', new_figure=True)
    

# Test the new model with new images

Testing the model trained above. Using a database i made, it checks that the fittings in unseen images make sense. There are asserts for mean error per image and the total error. 

In [ ]:
from os.path import isdir, join, isfile
from os import listdir
import numpy as np
import menpo.io as mio
from dlib import shape_predictor
from menpo.shape import PointCloud
from menpo.landmark import LandmarkGroup
from menpodetect import load_dlib_frontal_face_detector
from menpodetect import load_opencv_frontal_face_detector
from menpodetect.ffld2 import load_ffld2_frontal_face_detector
from menpodetect.dlib.conversion import pointgraph_to_rect
from menpofit.result import compute_normalise_point_to_point_error as comp_err
from menpofit.visualize import print_progress

def detection_to_pointgraph(detection):
    return PointCloud(np.array([(p.y, p.x) for p in detection.parts()]))

def compute_normalise_point_to_point_error_68(shape, gt_shape):
    normalizer = np.linalg.norm(gt_shape.points[36, :] - gt_shape.points[45, :])
    return comp_err(shape.points, gt_shape.points) / normalizer

def return_detector(detection):
    if detection == 'dlib':
        return load_dlib_frontal_face_detector()
    elif detection == 'opencv':
        from functools import partial
        det = load_opencv_frontal_face_detector()
        return partial(det, min_neighbours=3)
    elif detection == 'ffld2':
        return load_ffld2_frontal_face_detector()
    else:
        raise RuntimeError('Not a valid choice of detection ({}).'.format(detection))


In [ ]:
p0 = '/vol/atlas/homes/grigoris/Databases/personal/11_2015_test_localisation/'
method_landm_loc = 'dlibERT'
p_model = path_pickles + 'modelln_' + detection_method + '_dlibERT.model'
assert(isdir(p0) and isfile(p_model))
# detector = load_dlib_frontal_face_detector()
detector = return_detector(detection_method)

if method_landm_loc == 'dlibERT':
    predictor_dlib = shape_predictor(p_model)

images = list(mio.import_images(p0))
errors = []
for im in print_progress(images):
    im = im.crop_to_landmarks_proportion(0.3)
    detector(im, group_prefix='bb')
    l = list(im.landmarks.keys_matching('bb_*'))
    if len(l) == 0:
        print('No detection in {}.'.format(im.path.stem))
        continue
    assert(len(l) == 1)
    ln = im.landmarks['bb_0']
    if method_landm_loc == 'dlibERT':
        im_pili = np.array(im.as_PILImage())
        det_frame = predictor_dlib(im_pili, pointgraph_to_rect(ln.lms))
        init_pc = detection_to_pointgraph(det_frame)
    else:
        ft = model.fit_from_bb(im, ln.lms)
        init_pc = ft.final_shape
    ln1 = LandmarkGroup.init_with_all_label(init_pc)
    err = compute_normalise_point_to_point_error_68(ln1.lms, im.landmarks['PTS'].lms)
    assert(err < 0.3)
    errors.append(err)
mean_err = np.mean(np.array(errors))
assert(mean_err < 0.2)
print('Successfully predicted with mean error {}.'.format(mean_err))